In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
from pymc.math import ge, switch

%load_ext lab_black

# Equivalence of generic and brand-name drugs 

This is the second hypothesis testing example. The first is [here](https://areding.github.io/6420-pymc/unit6/Unit6-psoriasis.html).

Adapted from [Unit 6: equivalence.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit6/equivalence.odc).

Associated lecture video: Unit 6 lesson 7

## Problem Statement

The manufacturer wishes to demonstrate that their generic drug for a particular metabolic disorder is equivalent to a brand name drug. One of indication of the disorder is an abnormally low concentration of levocarnitine, an amino acid derivative, in the plasma. The treatment with the brand name drug substantially increases this concentration.

A small clinical trial is conducted with 43 patients, 18 in the Brand Name Drug arm and 25 in the Generic Drug arm. The increases in the log-concentration of levocarnitine are in the data below.

The FDA declares that bioequivalence among the two drugs can be established if the difference in response to the two drugs is within 2 units of log-concentration. Assuming that the log-concentration measurements follow normal distributions with equal population variance, can these two drugs be declared bioequivalent within a tolerance +/-2  units?

---
The way the data is set up in the .odc file is strange. It seems simpler to just have a separate list for each increase type.

In [2]:
# fmt: off
increase_type1 = [7, 8, 4, 6, 10, 10, 5, 7, 9, 8, 6, 7, 8, 4, 6, 10, 8, 9]
increase_type2 = [6, 7, 5, 9, 5, 5, 3, 7, 5, 10, 8, 5, 8, 4, 4, 8, 6, 11, 
                  7, 5, 5, 5, 7, 4, 6]
# fmt: on

We're using ```pm.math.switch``` and ```pm.math.eq``` to recreate the BUGS ```step()``` function for the ```probint``` variable.

See [Unit 6: Stress, Diet and Plasma Acids](./Unit6-stressacids.ipynb) to find out more about converting the BUGS step function.

In [3]:
with pm.Model() as m:
    # priors
    mu1 = pm.Normal("mu1", mu=10, sigma=316)
    mu2 = pm.Normal("mu2", mu=10, sigma=316)
    mudiff = pm.Deterministic("mudiff", mu1 - mu2)
    prec = pm.Gamma("prec", alpha=0.001, beta=0.001)
    sigma = 1 / pm.math.sqrt(prec)

    probint = pm.Deterministic(
        "probint",
        switch(ge(mudiff + 2, 0), 1, 0) * switch(ge(2 - mudiff, 0), 1, 0),
    )

    y_type1 = pm.Normal("y_type1", mu=mu1, sigma=sigma, observed=increase_type1)
    y_type2 = pm.Normal("y_type2", mu=mu2, sigma=sigma, observed=increase_type2)

    # start sampling
    trace = pm.sample(
        10000,
        chains=4,
        tune=1000,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/multipledispatch/dispatcher.py:27: AmbiguityWarning: 
Ambiguities exist in dispatched function _unify

The following signatures may result in ambiguous behavior:
	[ConstrainedVar, Var, Mapping], [object, ConstrainedVar, Mapping]
	[ConstrainedVar, Var, Mapping], [object, ConstrainedVar, Mapping]
	[object, ConstrainedVar, Mapping], [ConstrainedVar, object, Mapping]
	[ConstrainedVar, object, Mapping], [object, ConstrainedVar, Mapping]


Consider making the following additions:

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)
  warn(warning_text(dispatcher.name, ambiguities), AmbiguityWarning)
Multiprocess sampling (4 chains in 4

Sampling 4 chains for 1_000 tune and 10_000 draw iterations (4_000 + 40_000 draws total) took 14 seconds.


In [4]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu1,7.336,0.473,6.412,8.277,0.002,0.001,51503.0,31125.0,1.0
mu2,6.201,0.399,5.389,6.965,0.002,0.001,49352.0,31643.0,1.0
prec,0.263,0.058,0.155,0.380,0.000,0.000,43753.0,28900.0,1.0
mudiff,1.135,0.618,-0.059,2.378,0.003,0.002,51789.0,31031.0,1.0
probint,0.920,0.271,0.000,1.000,0.002,0.001,31350.0,31350.0,1.0


BUGS results:

|          | mean   | sd      | MC_error | val2.5pc | median | val97.5pc | start | sample |
|----------|--------|---------|----------|----------|--------|-----------|-------|--------|
| mu[1]    | 7.332  | 0.473   | 0.001469 | 6.399    | 7.332  | 8.264     | 1001  | 100000 |
| mu[2]    | 6.198  | 0.4006  | 0.001213 | 5.406    | 6.199  | 6.985     | 1001  | 100000 |
| mudiff   | 1.133  | 0.618   | 0.00196  | -0.07884 | 1.134  | 2.354     | 1001  | 100000 |
| prec     | 0.2626 | 0.05792 | 1.90E-04 | 0.1617   | 0.2584 | 0.3877    | 1001  | 100000 |
| probint  | 0.9209 | 0.2699  | 9.07E-04 | 0        | 1      | 1         | 1001  | 100000 |

In [5]:
%load_ext watermark
%watermark -n -u -v -iv -p pytensor

Last updated: Fri Feb 03 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

aesara: 2.8.10
aeppl : 0.1.1

matplotlib: 3.6.3
pymc      : 5.0.1
arviz     : 0.14.0
numpy     : 1.24.1

